In [2]:
import os
import re
import json
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


# Base path to the Factiva repository on the shared drive
base_folder = r"\\data2\CommercialData\Factiva_Repository"

# Step 1: Check if the base folder is accessible
if not os.path.exists(base_folder):
    print(f"Folder not accessible: {base_folder}")
    exit(1)  # Exit the script if the folder does not exist or access is denied
else:
    print(f"Access confirmed to: {base_folder}")

# Step 2: Initialize a list to store all extracted article records
all_records = []

# Step 3: Traverse all subdirectories and process each JSON file
for root, dirs, files in os.walk(base_folder):
    for filename in tqdm(files, desc=f"Processing JSON files in {root}"):
        if filename.endswith(".json"):  # Only process .json files
            file_path = os.path.join(root, filename)
            try:
                # Open and load the JSON file
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                    # Ensure the data is a list (even if only one article is present)
                    if isinstance(data, dict):
                        data = [data]

                    # Extract relevant fields from each article
                    for item in data:
                        record = {
                            "id": item.get("an"),
                            "title": item.get("title"),
                            "snippet": item.get("snippet"),
                            "body": item.get("body"),
                            "source_name": item.get("source_name"),
                            "publication_date": item.get("publication_date"),
                            "language": item.get("language_code"),
                            "word_count": item.get("word_count"),
                            "region": item.get("region_of_origin"),
                            "industry_codes": item.get("industry_codes"),
                            "subject_codes": item.get("subject_codes"),
                        }
                        all_records.append(record)
            except json.JSONDecodeError as e:
                print(f"JSON decode error in file {file_path}: {e}")
            except Exception as e:
                print(f"Unexpected error in file {file_path}: {e}")

# Step 4: Convert the list of records into a pandas DataFrame
df = pd.DataFrame(all_records)

# Print a summary: column names and number of articles
print("Columns: ", df.columns.tolist())
print("Number of articles: ", len(df))

Access confirmed to: \\data2\CommercialData\Factiva_Repository


Processing JSON files in \\data2\CommercialData\Factiva_Repository: 0it [00:00, ?it/s]
Processing JSON files in \\data2\CommercialData\Factiva_Repository\2020: 100%|█████████| 25/25 [02:58<00:00,  7.16s/it]
Processing JSON files in \\data2\CommercialData\Factiva_Repository\2021: 100%|█████████| 25/25 [03:18<00:00,  7.93s/it]
Processing JSON files in \\data2\CommercialData\Factiva_Repository\2022: 100%|█████████| 25/25 [04:01<00:00,  9.65s/it]
Processing JSON files in \\data2\CommercialData\Factiva_Repository\2023: 100%|█████████| 25/25 [04:32<00:00, 10.91s/it]
Processing JSON files in \\data2\CommercialData\Factiva_Repository\2024: 100%|█████████| 33/33 [04:20<00:00,  7.89s/it]
Processing JSON files in \\data2\CommercialData\Factiva_Repository\2005: 100%|█████████| 25/25 [00:00<00:00, 34.70it/s]
Processing JSON files in \\data2\CommercialData\Factiva_Repository\2006: 100%|█████████| 25/25 [00:50<00:00,  2.01s/it]
Processing JSON files in \\data2\CommercialData\Factiva_Repository\2007: 

Columns:  ['id', 'title', 'snippet', 'body', 'source_name', 'publication_date', 'language', 'word_count', 'region', 'industry_codes', 'subject_codes']
Number of articles:  19795095


In [3]:
df.shape

(19795095, 11)

In [4]:
# Step 5: Display the distribution of articles by language (as percentages)
language_distribution = df['language'].value_counts(normalize=True)

# Format as percentage and print
print((language_distribution * 100).round(2).astype(str) + ' %')

language
en       56.9 %
es      23.66 %
fr       9.47 %
zhcn     8.79 %
ar       1.18 %
Name: proportion, dtype: object


In [5]:
df_en = df[df['language'] == 'en']

In [6]:
df_en.shape

(11263164, 11)

In [7]:
df_es = df[df['language'] == 'es']

In [8]:
df_es.shape

(4683714, 11)

In [9]:
df_fr = df[df['language'] == 'fr']

In [10]:
df_fr.shape

(1874542, 11)

### Construct local and global Trade Policy Uncertainty (TPU) indices using text data from local and global news sources, drawing on the approaches of Caldara et al. (2018) and Bloom et al. (2016)

In [11]:
!pip install swifter

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import swifter
import time

# Enable progress bars for swifter and tqdm
tqdm.pandas()
swifter.config.progress_bar = True

class TPUDetector:
    def __init__(self):
        # Compile regex for trade-related terms
        self.trade_terms = re.compile(
            r"(USMCA|NAFTA|CUSMA|WTO|World Trade Organization|GATT|General Agreement on Tariffs and Trade|"
            r"Doha Round|Uruguay Round|trade polic(?:y|ies)|trade agreement(?:s)?|free trade(?: agreement(?:s)?)?|"
            r"FTA(?:s)?|preferential trade|bilateral trade|multilateral trade|trade negotiation(?:s)?|"
            r"trade act(?:s)?|trade treat(?:y|ies)|trade rule(?:s)?|trade friction(?:s)?|market access|"
            r"tariff(?:s)?|retaliatory tariff(?:s)?|retaliation|import tariff(?:s)?|export tariff(?:s)?|"
            r"tariff dut(?:y|ies)|custom(?:s)? dut(?:y|ies)|duty on import(?:s)?|import dut(?:y|ies)|"
            r"import barrier(?:s)?|import restriction(?:s)?|import liberalization|export restriction(?:s)?|"
            r"export subsid(?:y|ies)|\b(import|imports|imported|importing)\b|\b(export|exports|exported|exporting)\b|"
            r"border(?:s)?|trade barrier(?:s)?|non-tariff barrier(?:s)?|trade remed(?:y|ies)|"
            r"countervailing dut(?:y|ies)|trade dispute(?:s)?|trade panel(?:s)?|WTO ruling(?:s)?|"
            r"trade tribunal(?:s)?|trade retaliation(?:s)?|trade sanction(?:s)?|trade enforcement|protectionism|"
            r"unilateralism|trade liberalization|international trade|import (ban|tax|subsid)(?:es)?|"
            r"export (ban|tax|subsid)(?:es)?|border (ban|tax|subsid)(?:es)?|trade facilitation|escalating trade|"
            r"trade partnership(?:s)?|trade adjustment assistance|customs tariff(?:s)?|tariff preference(?:s)?|"
            r"trade restriction(?:s)?|trade embargo(?:es)?|import surcharge(?:s)?|sectoral tariff(?:s)?|"
            r"preferential tariff(?:s)?|reciprocal tariff(?:s)?|customs valuation rule(?:s)?|"
            r"import licensing requirement(?:s)?|rules of origin restriction(?:s)?|export control(?:s)?|"
            r"trade tax(?:es)?|import protection|protectionist barrier(?:s)?|plurilateral(?:s)?|"
            r"subsidies and countervailing measures|trade-restrictive|trade-facilitating|strategic tariff(?:s)?|"
            r"GATT ruling(?:s)?|WTO panel(?:s)?|GATT panel(?:s)?|WTO case(?:s)?|trade war(?:s)?|"
            r"customs union(?:s)?|anti-dumping)",
            flags=re.IGNORECASE
        )

        # Compile regex for uncertainty-related terms
        self.uncertainty_terms = re.compile(
            r"(uncertain(?:ty|ties)?|unpredictabl(?:e|ility)?|volatil(?:e|ity)|downside risk|upside risk|unexpected|"
            r"unknown|crisis|crises|war|unclear|tension(?:s)?|danger(?:s)?|fear(?:s)?|concern(?:s|ed| about)?|"
            r"caution|worr(?:y|ies)?|anxious|anxiety|unease|unstabl(?:e|ity)|threat(?:s)?|threaten(?:s|ed|ing)?|"
            r"ambiguous|ambiguity|imprecise|vague|unresolved|unanticipated|unforeseen|hesitant|hesitation|"
            r"doubt(?:ful|s)?|skeptic(?:al|ism)?|murky|precarious|tentative|fluid|chang(?:eable|ing)|shifting|"
            r"wavering|turmoil|turbulent|turbulence|fragil(?:e|ity)|fluctuation(?:s)?|slowdown|downturn|"
            r"depression|recession(?:ary)?|pessimism|pessimistic|stagflation|erosion|deterioration|meltdown|"
            r"bubble burst|stress(?:ed)?|distress|vulnerab(?:le|ility|ilities)?|apprehensive|possibilit(?:y|ies)?|"
            r"likelihood|probabilit(?:y|ies)?|prospect(?:s)?|potential|speculat(?:ion|ive)|rumor(?:s)?|"
            r"rumours?|bleak|gloom|nervousness|cautious|wary|unconfirmed|pressure(?:s)?|confusion|"
            r"challenge\w*|dispute(?:s)?|issue(?:s)?|dubious)",
            flags=re.IGNORECASE
        )

        # Combined pattern: trade and uncertainty terms within 10 words of each other
        self.tpu_pattern = re.compile(
            rf"({self.trade_terms.pattern}(?:\W+\w+){{1,10}}{self.uncertainty_terms.pattern})|"
            rf"({self.uncertainty_terms.pattern}(?:\W+\w+){{1,10}}{self.trade_terms.pattern})",
            flags=re.IGNORECASE
        )

    def normalize_text_preserving_acronyms(self, text: str) -> str:
        """Clean text by removing punctuation and lowering case, while preserving acronyms (e.g., IMF, WTO)."""
        if not isinstance(text, str):
            return ""
        # Identify acronyms (e.g., IMF, WTO) and temporarily replace them
        acronyms = re.findall(r'\b[A-Z]{2,}\b', text)
        for i, ac in enumerate(acronyms):
            text = text.replace(ac, f"__ACRO_{i}__")
        text = re.sub(r"[^\w\s]", " ", text).lower()
        for i, ac in enumerate(acronyms):
            text = text.replace(f"__acro_{i}__", ac)
        return text

    def detect_tpu(self, text: str) -> bool:
        """Return True if both trade and uncertainty terms co-occur within a 10-word window."""
        return bool(self.tpu_pattern.search(str(text)))

    def apply_to_dataframe(self, df: pd.DataFrame, text_column: str = "body") -> pd.DataFrame:
        """Clean the text column, apply TPU detection, and return updated DataFrame."""
        df = df.copy()
        df[text_column] = df[text_column].fillna('')
        df["body_clean"] = df[text_column].apply(self.normalize_text_preserving_acronyms)

        print("Running TPU detection with swifter...")
        start = time.time()
        df["tpu_flag"] = df["body_clean"].swifter.apply(self.detect_tpu).astype(int)
        print(f"Completed in {round(time.time() - start, 2)} seconds.")
        return df

In [ ]:
# Initialize the TPU detector
tpu_detector = TPUDetector()

# Apply it to your filtered trade-related articles
df_en_with_tpu = tpu_detector.apply_to_dataframe(df_en, text_column="body")

In [ ]:
# Convert timestamp (in milliseconds) to a readable date
df_en_with_tpu['pub_date_readable'] = pd.to_datetime(df_en_with_tpu['publication_date'], unit='ms')

# Extract year and month from the publication date
df_en_with_tpu['pub_year'] = df_en_with_tpu['pub_date_readable'].dt.year
df_en_with_tpu['pub_month'] = df_en_with_tpu['pub_date_readable'].dt.month

# Sort articles by publication date
df_en_with_tpu.sort_values(by='pub_date_readable', inplace=True)

In [ ]:
# Step 1: Add a monthly period column based on publication date
df_en_with_tpu['pub_period'] = df_en_with_tpu['pub_date_readable'].dt.to_period('M')

# Step 2: Define a function to compute the TPU index by time period (or any group)
def compute_tpu_index_grouped(df, group_cols=['pub_date_readable'], text_col='body_clean', flag_col='tpu_flag', freq='M'):
    df = df.copy()

    # Convert to datetime if grouping by date column
    if 'pub_date_readable' in group_cols:
        df['pub_date_readable'] = pd.to_datetime(df['pub_date_readable'])
        if freq:
            # Convert to period, then back to timestamp for grouping
            df['period'] = df['pub_date_readable'].dt.to_period(freq).dt.to_timestamp()
            group_cols = ['period'] + [col for col in group_cols if col != 'pub_date_readable']

    # Group by selected columns and compute total articles and TPU-flagged articles
    grouped = df.groupby(group_cols).agg(
        NUMBER_ARTICLES=(text_col, 'count'),
        TPUD_ARTICLES=(flag_col, 'sum')
    ).reset_index()

    # Compute TPU index as percentage
    grouped['TPUD_index'] = (grouped['TPUD_ARTICLES'] / grouped['NUMBER_ARTICLES'] * 100).round(2)

    # Ensure article counts are integer type
    grouped['NUMBER_ARTICLES'] = grouped['NUMBER_ARTICLES'].astype(int)
    grouped['TPUD_ARTICLES'] = grouped['TPUD_ARTICLES'].astype(int)

    return grouped

# Step 3: Compute the monthly TPU index using df_en_with_tpu
monthly_tpu_all = compute_tpu_index_grouped(
    df=df_en_with_tpu,
    group_cols=['pub_date_readable'],
    text_col='body_clean',
    flag_col='tpu_flag',
    freq='M'  # monthly frequency
)

In [ ]:
# Ensure the period column is in datetime format for plotting
monthly_tpu_all['pub_month_period'] = pd.to_datetime(monthly_tpu_all['pub_month_period'])

# Scale the TPU index (multiply by 100) for better percentage-based display
monthly_tpu_all['TPUD_index_scaled'] = monthly_tpu_all['TPUD_index'] * 100

# Create the figure and axis
fig, ax = plt.subplots(figsize=(14, 6))

# Plot the TPU index over time using a clean line style
ax.plot(
    monthly_tpu_all['pub_month_period'],
    monthly_tpu_all['TPUD_index_scaled'],
    marker='o',
    linestyle='-',
    linewidth=2,
    color='navy',
    label='TPU Index'
)

# Add title and axis labels
ax.set_title('Factiva - Global Trade Policy Uncertainty Index', fontsize=16, fontweight='bold')
ax.set_xlabel('Month', fontsize=13)
ax.set_ylabel('GTPU Index', fontsize=13)

# Remove gridlines for a cleaner look
ax.grid(False)

# Format x-axis to show one tick every 12 months
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Improve layout and spacing
plt.tight_layout()

# Add legend (useful even for a single line for formal completeness)
ax.legend(loc='upper left', fontsize=12)

# Remove top and right borders for a cleaner visual
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Display the plot
plt.show()

### Filter English-language articles related to trade using keyword matching

In [18]:
class TradeArticleFilter:
    def __init__(self):
        # Define the list of trade-related keywords (with regex for plural/singular variants)
        self.keywords = [

            # Basic trade terms
            r"import(?:s|ing)?", r"export(?:s|ing)?", r"export(?:ing)? market(?:s)?", r"export competitiven(?:ess)?",
            r"importing", r"exporting", r"trading", r"trade(?:s|d|ing)?", r"commerce", r"global trade", r"import licence(?:s)?",
            r"goods trade", r"service(?:s)? trade", r"services trade", r"GTA",

            # Tariffs and duties
            r"tariff(?:s)?", r"tariff(?:s)? hike", r"tariff(?:s)? increase", r"tariff(?:s)? cut",
            r"tariff(?:s)? exemption", r"retaliatory tariff(?:s)?", r"tariff quota(?:s)?", r"tariff binding",
            r"ad valorem tariff(?:s)?", r"border tax adjustment", r"countervailing dut(?:y|ies)?",
            r"customs dut(?:y|ies)", r"custom duties", r"import dut(?:y|ies)", r"export dut(?:y|ies)", r"import tax(?:e|es)?", 
            r"countervailing", r"countervailing duty",

            # Trade agreements
            r"free trade agreement(?:s)?", r"bilateral trade agreement(?:s)?", r"multilateral trade agreement(?:s)?",
            r"preferential trade agreement(?:s)?", r"regional trade agreement(?:s)?", r"trade treaty(?:ies)?",
            r"trade agreement(?:s)?", r"FTA(?:s)?", r"NAFTA", r"CUSMA", r"USMCA", r"CAFTA",

            # Institutions and mechanisms
            r"WTO(?: dispute| ruling| panel)?", r"WTO negotiation(?:s)?", r"World Trade Organization",
            r"wto dispute", r"GATT", r"Doha round", r"Uruguay round",

            # Barriers and restrictions
            r"non-tariff barrier(?:s)?", r"non-tariff measure(?:s)?", r"nontariff measure(?:s)?",
            r"technical trade barrier(?:s)?", r"import ban(?:s)?", r"export ban(?:s)?",
            r"export barrier(?:s)?", r"import barrier(?:s)?", r"trade embargo(?:es)?", r"quotas?",
            r"import quota(?:s)?", r"export quota(?:s)?", r"binding quota(?:s)?", r"non-binding quota(?:s)?",
            r"safeguard measure(?:s)?", r"rules of origin", r"local content", r"local content requirement(?:s)?",
            r"voluntary export restraint(?:s)?", r"voluntary export", r"voluntary export restraint arrangements",
            r"voluntary import expansion", r"tariff quota", r"trade-related investment measure(?:s)?",
            r"export credits?", r"export control(?:s)?", r"customs (?:procedure|reform|clearance)",
            r"customs enforcement",

            # Trade policy
            r"foreign trade policy", r"trade policy", r"trade policy uncertainty", r"uncertain trade environment",
            r"protectionist polic(?:y|ies)", r"liberalization polic(?:y|ies)?", r"trade liberalization",
            r"policy reversal(?:s)?", r"policy backtracking", r"trade spillover(?:s)?",

            # Supply chains and logistics
            r"supply chain(?: disruption(?:s)?| shock(?:s)?| bottleneck(?:s)?| resilience| pressure(?:s)?)?",
            r"supply[- ]chain(?: disruption(?:s)?| shock(?:s)?| bottleneck(?:s)?| resilience| pressure(?:s)?)?",
            r"global value chain(?:s)?", r"GVC(?:s)?", r"port congestion", r"shipping delay(?:s)?",
            r"container shortage(?:s)?", r"logistics disruption(?:s)?", r"reshoring", r"nearshoring",
            r"friend-shoring", r"cargo",

            # Geopolitical factors
            r"geopolitical tension(?:s)?", r"geopolitical shock(?:s)?", r"geopolitical concern(?:s)?",
            r"geopolitical fragmentation", r"geo-economic fragmentation", r"geoeconomic fragmentation",
            r"geoeconomic", r"geopolitical", r"geopolitics", r"economic fragmentation",
            r"strategic competitiveness", r"economic coercion", r"trade war(?:s)?", r"trade tension(?:s)?",
            r"trade disruption(?:s)?", r"fragmented trade",

            # External sector
            r"external sector", r"FX intervention(?:s)?", r"foreign exchange intervention(?:s)?",
            r"foreign exchange market", r"foreign exchange polic(?:y|ies)?",
            r"international reserve(?:s)?", r"foreign exchange reserve(?:s)?", r"foreign asset(?:s)?",
            r"real exchange rate", r"current account",

            # Other macro indicators
            r"commodity price (?:shock|surge)", r"commodity export ban(?:s)?",
            r"trade balance", r"trade deficit", r"net-commodity-importing", r"FDI measures",
        ]

        # Compile regex pattern once for performance
        self.pattern = re.compile(r"(?:{})".format("|".join(self.keywords)), flags=re.IGNORECASE)

    def filter(self, df, text_column="snippet"):
        """Filter rows in a DataFrame where the text_column contains trade-related content."""
        df_filtered = df[df[text_column].str.contains(self.pattern, na=False)].copy()
        df_filtered.reset_index(drop=True, inplace=True)
        return df_filtered


In [ ]:
# Initialize the filter
trade_filter = TradeArticleFilter()

# Apply it to English articles DataFrame (df_en)
df_trade = trade_filter.filter(df_en, text_column="snippet")

In [19]:
df_trade.shape 

(1775175, 11)

In [21]:
df_trade.head(1)

,id,title,snippet,body,source_name,publication_date,language,word_count,region,industry_codes,subject_codes
0,KYRGYZE020200121eg1h00015,Copper leads price hike in non-ferrous base me...,Copper climbed to an eight-month high on Thurs...,“I'm cautiously optimistic on the outlook for ...,Kyrgyzstan Newsline,1579219200000,en,417,ASIA CASIAZ KIRGH USSRZ,,",gtrade,e51,ecat,gcat,gdip,gpir,"
1,OMDLOB0020200106eg150000g,"Second phase targets policies, capabilities",The Sultanate is making efforts to achieve str...,"According to Sami bin Salem al Saheb, Director...",Oman Daily Observer,1578182400000,en,481,ASIA GULFST MEASTZ OMAN WASIAZ,,",gvbod,c13,ccat,gcat,gpir,gpol,ncat,nfact,nfcpin,"


In [22]:
df_trade.tail(1)

,id,title,snippet,body,source_name,publication_date,language,word_count,region,industry_codes,subject_codes
1775173,NBFSTA0020250129el1t0000k,Jamaat-e-Islami Emir Hafiz Naeem ur Rehman ann...,Jamaat-e-Islami Emir Hafiz Naeem ur Rehman has...,He hailed the return of millions of Gazans to ...,Frontier Star,1738108800000,en,542,ASIA PAKIS SASIAZ,",i1,i16,ieutil,iutil,",",c314,ccat,cdom,gcat,gcivds,gcns,gcom,gpir,gpo..."
1775174,ECOWEK0020250117el1h000fx,Economics; Researcher from Mersin Details Find...,2025 JAN 17 (VerticalNews) -- By a News Report...,Our news journalists obtained a quote from the...,Economics Week,1737072000000,en,422,NAMZ USA,",i43,icargo,iclt,icnp,itsp,",",c23,ccat,cscm,gcat,genv,gsust,"


In [71]:
df_trade.shape

(21296, 12)

In [75]:
#df_trade.to_pickle(r"E:\data\FAyivodji\data\factiva_df_trade_news.pkl")